In [88]:
import pandas as pd
from pathlib import Path
import streamlit as st
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [89]:
file_path = Path("diabetes_hospitalizations_unedited.csv")
hospitalizations_df = pd.read_csv(file_path)
df.head()

,id,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
0,1,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,...,No,No,No,No,No,No,No,No,No,NO
1,2,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,3,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,...,No,No,No,No,No,No,No,No,Yes,NO
3,4,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,5,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [90]:
hospitalizations_df.set_index('encounter_id').head()

,id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
encounter_id,,,,,,,,,,,,,,,,,,,,,
2278392,1,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
149190,2,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
64410,3,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
500364,4,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
16680,5,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [91]:
hospitalizations_df = hospitalizations_df.replace('Steady', '1')
#hospitalizations_df.replace('No', '0')
#hospitalizations_df.replace('Up', 1)
#hospitalizations_df.replace('Down', 1)

In [92]:
hospitalizations_df = hospitalizations_df.replace('No', '0')
hospitalizations_df

,id,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
0,1,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,...,0,0,0,0,0,0,0,0,0,NO
1,2,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,...,0,Up,0,0,0,0,0,Ch,Yes,>30
2,3,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,...,0,0,0,0,0,0,0,0,Yes,NO
3,4,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,...,0,Up,0,0,0,0,0,Ch,Yes,NO
4,5,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,...,0,1,0,0,0,0,0,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,101762,443847548,100162476,AfricanAmerican,Male,[70-80),?,1,3,7,...,0,Down,0,0,0,0,0,Ch,Yes,>30
101762,101763,443847782,74694222,AfricanAmerican,Female,[80-90),?,1,4,5,...,0,1,0,0,0,0,0,0,Yes,NO
101763,101764,443854148,41088789,Caucasian,Male,[70-80),?,1,1,7,...,0,Down,0,0,0,0,0,Ch,Yes,NO
101764,101765,443857166,31693671,Caucasian,Female,[80-90),?,2,3,7,...,0,Up,0,0,0,0,0,Ch,Yes,NO


In [93]:
hospitalizations_df = hospitalizations_df.replace('Down', '1')

In [94]:
hospitalizations_df = hospitalizations_df.replace('Up', '1')

In [95]:
hospitalizations_df = hospitalizations_df.drop(columns='patient_nbr')


In [96]:
hospitalizations_df = hospitalizations_df.drop(columns='medical_specialty')
#'number_diagnoses', 'number_inpatient', 'num_medications', 'num_procedures',
                                   # 'patient_nbr',)


In [97]:
hospitalizations_df = hospitalizations_df.drop(columns='diag_1')

In [98]:
hospitalizations_df = hospitalizations_df.drop(columns='max_glu_serum')

In [99]:
hospitalizations_df = hospitalizations_df.drop(columns='diag_2')

In [100]:
hospitalizations_df = hospitalizations_df.drop(columns='diag_3')

In [101]:
hospitalizations_df = hospitalizations_df.drop(columns='admission_source_id')

In [102]:
hospitalizations_df = hospitalizations_df.drop(columns='discharge_disposition_id')

In [103]:
hospitalizations_df = hospitalizations_df.drop(columns='payer_code')

In [104]:
hospitalizations_df = hospitalizations_df.drop(columns='weight')

In [105]:
hospitalizations_df = hospitalizations_df.drop(columns='admission_type_id')

In [106]:
hospitalizations_df = hospitalizations_df.drop(columns='num_lab_procedures')

In [107]:
hospitalizations_df = hospitalizations_df.drop(columns='encounter_id')

In [108]:
hospitalizations_df = hospitalizations_df.drop(columns='number_outpatient')

In [109]:
hospitalizations_df = hospitalizations_df.drop(columns='number_emergency')

In [110]:
#hospitalizations_df = hospitalizations_df.drop(columns=')

In [111]:
hospitalizations_df.head()

,id,race,gender,age,time_in_hospital,num_procedures,num_medications,number_inpatient,number_diagnoses,A1Cresult,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
0,1,Caucasian,Female,[0-10),1,0,1,0,1,None,...,0,0,0,0,0,0,0,0,0,NO
1,2,Caucasian,Female,[10-20),3,0,18,0,9,None,...,0,1,0,0,0,0,0,Ch,Yes,>30
2,3,AfricanAmerican,Female,[20-30),2,5,13,1,6,None,...,0,0,0,0,0,0,0,0,Yes,NO
3,4,Caucasian,Male,[30-40),2,1,16,0,7,None,...,0,1,0,0,0,0,0,Ch,Yes,NO
4,5,Caucasian,Male,[40-50),1,0,8,0,5,None,...,0,1,0,0,0,0,0,Ch,Yes,NO


In [112]:
hospitalizations_df=hospitalizations_df.drop(columns='change')

In [113]:
hospitalizations_df = hospitalizations_df.drop(columns='diabetesMed')

In [114]:
hospitalizations_df = hospitalizations_df.drop(columns='readmitted')

In [115]:
hospitalizations_df = hospitalizations_df.set_index('id')

In [116]:
hospitalizations_df.head()

,race,gender,age,time_in_hospital,num_procedures,num_medications,number_inpatient,number_diagnoses,A1Cresult,metformin,...,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone
id,,,,,,,,,,,,,,,,,,,,,
1,Caucasian,Female,[0-10),1,0,1,0,1,None,0,...,0,0,0,0,0,0,0,0,0,0
2,Caucasian,Female,[10-20),3,0,18,0,9,None,0,...,0,0,0,0,1,0,0,0,0,0
3,AfricanAmerican,Female,[20-30),2,5,13,1,6,None,0,...,0,0,0,0,0,0,0,0,0,0
4,Caucasian,Male,[30-40),2,1,16,0,7,None,0,...,0,0,0,0,1,0,0,0,0,0
5,Caucasian,Male,[40-50),1,0,8,0,5,None,0,...,0,0,0,0,1,0,0,0,0,0


In [119]:
hospitalizations_df.to_csv('hospitalizations_dm_cleaned.csv')

id
1     0-10)
2    10-20)
3    20-30)
4    30-40)
5    40-50)
Name: age, dtype: object